In [3]:
import requests
from lxml import etree
import csv
from datetime import datetime, timedelta

dm = [57519,54431]  # 假设有多个城市代码
start_datatimee = '2021-09-26'
fin_datatimee = '2021-10-27'

start_date = datetime.strptime(start_datatimee, '%Y-%m-%d')
end_date = datetime.strptime(fin_datatimee, '%Y-%m-%d')

def check_data_available(date):
    """检查指定日期是否有数据"""
    url = url_template.format(date=date)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"请求失败，状态码: {response.status_code}")
        return False
    return "无历史数据" not in response.text

def find_earliest_date(start_date):
    current_date = datetime.strptime(start_date, '%Y-%m-%d')

    # 1. 查找最早年份
    while True:
        test_date = current_date.replace(year=current_date.year - 1)
        if check_data_available(test_date.strftime('%Y-%m-%d')):
            current_date = test_date
        else:
            break

    # 2. 查找最早月份
    while True:
        if current_date.month == 1:  # 如果是 1 月，跳到上一年的 12 月
            test_date = current_date.replace(year=current_date.year - 1, month=12)
        else:  # 否则，直接减少一个月
            test_date = current_date.replace(month=current_date.month - 1)

        if check_data_available(test_date.strftime('%Y-%m-%d')):
            current_date = test_date
        else:
            break

    # 3. 在当前年月内精确到日期，从当月的最后一天开始向前推
    last_day_of_month = current_date.replace(day=1) + timedelta(days=31)
    last_day_of_month = last_day_of_month.replace(day=1) - timedelta(days=1)
    
    current_date = last_day_of_month
    while current_date >= last_day_of_month.replace(day=1):  # 确保仍在当前月份内
        if not check_data_available(current_date.strftime('%Y-%m-%d')):  # 找到无数据日期
            current_date += timedelta(days=1)  # 后一天即为最早有数据日期
            return current_date
        else:
            current_date -= timedelta(days=1)

    # 如果月份内所有日期都有数据，则返回当月第一天
    return current_date


for city in dm:
    url_template = f"https://q-weather.info/weather/{city}/history/?date={{date}}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62',
    }
    # 找到最早的日期
    earliest_date = find_earliest_date(start_datatimee)
    print(f"{city}最早有数据的日期为: {earliest_date.strftime('%Y-%m-%d')}")
    # 生成文件名：使用城市代码和日期段
    filename = f"weather_{city}_{start_datatimee}_to_{fin_datatimee}.csv"
    
    # 写入 CSV 文件的表头（如果文件不存在，则创建）
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['城市', '时次', '瞬时温度', '地面气压', '相对湿度', '瞬时风向', 
                         '瞬时风速', '1小时极大风速', '1小时降水', '10分钟平均能见度'])
    
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime('%Y-%m-%d')
        url = f"https://q-weather.info/weather/{city}/history/?date={formatted_date}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62',
        }
        response = requests.get(url, headers=headers)
        html = response.text
        tree = etree.HTML(html)
        
        # 定位到数据行
        rows = tree.xpath('//*[@align="center"]')
        for row in rows:
            # 提取各列数据，并处理空值
            data = [city]  # 包含城市和日期
            for idx in range(9):  # 假设有 9 列
                try:
                    cell = row.xpath(f'./td[{idx + 1}]/text()')
                    data.append(cell[0] if cell else None)  # 如果为空则填充 None
                except IndexError:
                    data.append(None)  # 如果索引超出范围也填充 None

            # 将数据追加写入对应城市的 CSV 文件
            with open(filename, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(data)

        current_date += timedelta(days=1)


57519最早有数据的日期为: 2013-11-14
54431最早有数据的日期为: 2013-11-14
